In [1]:
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

### 3. 지도 시각화

#### 3.1 데이터 로딩 및 전처리

In [2]:
import pandas as pd
raw_total = pd.read_excel('./files/1_crawling_raw.xlsx')
raw_total.head()

content        data like   place  \
0  제주도 도착~~공항근처 #제주분식 에서 먹은 첫식사만족만족!#제주맛집#제주도맛집#제...  2019-01-01  705     NaN   
1  ﻿#함덕맛집 무거버거 바다 보면서 먹으니까 더 꿀맛👍🏻 #제주도카페투어#제주일상#함...  2019-01-01  384     NaN   
2  .대만족 제주스!.#동문시장#제주동문시장#제주도기념품#제주기념품#제주스는완벽#제주도...  2019-01-01  595     NaN   
3  #제주여행#2일차무거버거 제주와서 먹은것중에 제일 맛있었음!#제주도여행#제주맛집#함...  2019-01-01  412     NaN   
4  🥇1월 영업 안내🥇-1월 2일, 3일(수,목)은 쉬어갑니다-1월 4일,5일,6일(금...  2019-01-01  364  로타리과자점   

                                                tags  
0  ['#제주분식', '#제주맛집', '#제주도맛집', '#제주맛집추천', '#제주도맛...  
1  ['#함덕맛집', '#제주도카페투어', '#제주일상', '#함덕', '#jejudo...  
2  ['#동문시장', '#제주동문시장', '#제주도기념품', '#제주기념품', '#제주...  
3  ['#제주여행', '#2일차무거버거', '#제주도여행', '#제주맛집', '#함덕맛...  
4                                                 []

In [3]:
# get location counts
location_counts = raw_total['place'].value_counts()
location_counts

Jeju                               241
Jeju-do                            172
Jungle Book by Alice               101
Seogwipo                            65
제주도 크리스마스 박물관                       59
                                  ... 
섭지코지 (Seopjikoji)                    1
우도 (Udo / 牛島)                        1
돈사촌                                  1
나주혁신도시                               1
산방굴사 (Sanbanggulsa Temple 山房窟寺)      1
Name: place, Length: 1027, dtype: int64

In [4]:
#make into a dataframe and save
location_counts_df = pd.DataFrame(location_counts)
location_counts_df.head()

place
Jeju                    241
Jeju-do                 172
Jungle Book by Alice    101
Seogwipo                 65
제주도 크리스마스 박물관            59

In [5]:
location_counts_df.to_excel('./files/3_location_counts.xlsx')

In [6]:
locations = list(location_counts.index)
locations

['Jeju',
 'Jeju-do',
 'Jungle Book by Alice',
 'Seogwipo',
 '제주도 크리스마스 박물관',
 '제주에인감귤밭',
 'Jeju Island',
 '성산일출봉 城山日出峰  Seongsan Ilchulbong',
 '폼포코식당_pompokokitchen',
 '1100고지',
 'Nimome',
 '월정리해변',
 '석부작박물관',
 '제주도 애월읍',
 '알뜨르 비행장',
 '카멜리아 힐',
 '밥깡패',
 '제주 함덕 서우봉 해변',
 '제주레포츠랜드',
 '고집돌우럭중문점',
 '제주도 서귀포 중문관광단지 濟州島西歸浦中文觀光團地',
 '할로비치',
 '제주신화월드 Jeju Shinhwa World',
 '제주고궁한복카페 jeju gogung hanbok studio',
 '위미동백나무군락지',
 'Woljeongri Beach',
 '연동 바오젠 거리',
 '협재해변 Beach',
 '새별오름',
 '고래배꼽',
 '용머리해안',
 '용눈이오름',
 '제주빅볼랜드',
 '사려니숲길',
 '하이엔드 제주',
 '김녕미로공원 Jeju Kimnyoung Maze Park',
 '캔디원',
 'Jeju Island 제주특별자치도 济州道',
 '제주커피박물관 Baum',
 '위미동백나무군락',
 '삼무공원',
 'Jeju Aewol',
 '위미2리 동백군락지',
 '서귀포 느영나영 게스트하우스',
 '한라산',
 '마마뜰',
 '휴애리 자연생활공원',
 '광치기해변',
 '서우봉',
 '한라산 (漢拏山, Hallasan)',
 '제주해남',
 '옹포별장가든',
 '제주어린왕자게스트하우스',
 '월정리카페콧수염',
 '성이시돌목장',
 '제주 송악산',
 '카페브리프',
 '곽지해수욕장',
 '제주 꿈꾸는고래 스쿠버&게스트하우스',
 '아줄레주',
 '카페한라산',
 '牛岛 Udo Island 우도',
 '산굼부리',
 '에코랜드',
 '고집돌우럭-gozipfish',
 '소세지가게',
 '비자림',
 '제주 협재 해수욕장'

#### 3.2 카카오 API 활용, 위치 정보 획득

In [7]:
# use kakao open API to get address and x, y coordinates
import requests

searching = "합정 스타벅스"
url = "https://dapi.kakao.com/v2/local/search/keyword.json?query={}".format(searching)

headers = {
    "Authorization": "KakaoAK bdd6b9af0b72bfb5a5aeb3ae89948a72"
}

places = requests.get(url, headers = headers).json()['documents']
places

[{'address_name': '서울 마포구 서교동 395-166',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '26572121',
  'phone': '',
  'place_name': '스타벅스 서교점',
  'place_url': 'http://place.map.kakao.com/26572121',
  'road_address_name': '서울 마포구 양화로 78',
  'x': '126.916980454434',
  'y': '37.5514601750423'},
 {'address_name': '서울 마포구 합정동 472',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '2057327896',
  'phone': '1522-3232',
  'place_name': '스타벅스 합정점',
  'place_url': 'http://place.map.kakao.com/2057327896',
  'road_address_name': '서울 마포구 월드컵로1길 14',
  'x': '126.91253700818196',
  'y': '37.54994959743763'},
 {'address_name': '서울 마포구 서교동 490',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '288597324',
  'phone': '',
  'place_name': '스타벅스 합정폴리스R

In [8]:
# function for getting place name, and x, y coordinates
def find_places(searching):
    # get url
    url = url = "https://dapi.kakao.com/v2/local/search/keyword.json?query={}".format(searching)
    
    #include header
    headers = {
    "Authorization": "KakaoAK bdd6b9af0b72bfb5a5aeb3ae89948a72"
    }
    
    # request API
    places = requests.get(url, headers = headers).json()['documents']
    
    # select desired info
    place = places[0]
    name = place['place_name']
    x = place['x']
    y = place['y']
    data = [name, x, y, searching]
    
    return data

In [9]:
data = find_places('제주공항')
data

['제주국제공항', '126.492769004244', '33.5070789578184', '제주공항']

In [10]:
#iterate through locations
# tqdm shows progress of the iteration
import time
from tqdm import tqdm
locations_inform = []
for location in tqdm(locations):
    try:
        data = find_places(location)
        locations_inform.append(data)
        time.sleep(0.5)
    except:
        pass
locations_inform

100%|██████████████████████████████████████████████████████████████████████████████| 1027/1027 [08:38<00:00,  1.98it/s]


[['Jeju in aA', '126.83650369902168', '33.53833652360498', 'Jeju'],
 ['서귀포잠수함', '126.558616052674', '33.2393033784206', 'Seogwipo'],
 ['바이나흐튼 크리스마스박물관', '126.32785786016811', '33.2915720831759', '제주도 크리스마스 박물관'],
 ['제주에인감귤밭', '126.539409663514', '33.2566099433115', '제주에인감귤밭'],
 ['제주도', '126.545875710283', '33.3797769227161', 'Jeju Island'],
 ['한라산1100고지', '126.462219691112', '33.3580781709788', '1100고지'],
 ['월정리해수욕장', '126.795805057888', '33.556469394054', '월정리해변'],
 ['석부작박물관', '126.536501983154', '33.2504031491774', '석부작박물관'],
 ['올레길 17코스(광령-제주원도심 올레)', '126.439341083568', '33.4608856490895', '제주도 애월읍'],
 ['알뜨르비행장', '126.271527320164', '33.2047161778831', '알뜨르 비행장'],
 ['카멜리아힐', '126.370009467525', '33.2896344311608', '카멜리아 힐'],
 ['밥깡패', '126.263436208895', '33.410093302084', '밥깡패'],
 ['함덕해수욕장', '126.669238934013', '33.5430615661113', '제주 함덕 서우봉 해변'],
 ['제주레포츠랜드', '126.6387551242765', '33.48206819907232', '제주레포츠랜드'],
 ['고집돌우럭 중문점', '126.416704762779', '33.2579811121134', '고집돌우럭중문점'],
 

In [11]:
#save info
locations_inform_df = pd.DataFrame(locations_inform)
locations_inform_df.columns = ['카카오위치명', '경도', '위도', '인스타위치명']
locations_inform_df.to_excel('./files/3_locations.xlsx', index = False)

In [12]:
location_counts_df = pd.read_excel("./files/3_location_counts.xlsx", index_col = 0)
location_inform_df = pd.read_excel("./files/3_locations.xlsx")

In [13]:
# merge two data
location_data = pd.merge(location_inform_df, location_counts_df, how ='inner', left_on = '카카오위치명', right_index=True)
location_data.head()

카카오위치명          경도         위도       인스타위치명  place
1     서귀포잠수함  126.558616  33.239303     Seogwipo      1
351   서귀포잠수함  126.558616  33.239303       서귀포잠수함      1
3    제주에인감귤밭  126.539410  33.256610      제주에인감귤밭     48
4        제주도  126.545876  33.379777  Jeju Island      4
86       제주도  126.545876  33.379777          제주도      4

In [14]:
# look for duplicates
location_data['카카오위치명'].value_counts()

자매국수 본점    3
오설록티뮤지엄    3
구좌상회작업실    2
제주도        2
제주돌마을공원    2
          ..
이런날엔       1
닭머르        1
바다이야기      1
포팩토리       1
한라산        1
Name: 카카오위치명, Length: 398, dtype: int64

In [15]:
#use pivot to sum up counts for duplicates
location_data = location_data.pivot_table(
    index = ['카카오위치명', '경도', '위도'],
    values = 'place',
    aggfunc = 'sum')

location_data = location_data.reset_index()
location_data.head()

카카오위치명          경도         위도  place
0  702키친  126.534134  33.498835      1
1   73st  126.455109  33.456989      1
2   가드망제  126.532954  33.500991      1
3     가배  126.505570  33.231541      1
4   가시식당  126.771438  33.353060      2

In [16]:
# save data 
location_data.to_excel("./files/3_locations_inform.xlsx", index = False)

In [17]:
# read data again
location_data = pd.read_excel("./files/3_locations_inform.xlsx")
location_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 4 columns):
카카오위치명    399 non-null object
경도        399 non-null float64
위도        399 non-null float64
place     399 non-null int64
dtypes: float64(2), int64(1), object(1)
memory usage: 12.6+ KB


#### 3.3 데이터 시각화

In [18]:
# plot the locations

import folium
Mt_Hanla = [33.362500, 126.533694]
map_jeju = folium.Map(location = Mt_Hanla, zoom_start = 11)

for i in range(len(location_data)):
    name = location_data['카카오위치명'][i]
    count = location_data['place'][i]
    size = int(count) * 2
    long = float(location_data['위도'][i])
    lat = float(location_data['경도'][i])
    folium.CircleMarker((long, lat), radius = size, color = 'red', popup=name).add_to(map_jeju)
    
map_jeju

In [19]:
# put markers into clusters
from folium.plugins import MarkerCluster

locations = []
names = []

for i in range(len(location_data)):
    data = location_data.iloc[i]
    locations.append((float(data['위도']), float(data['경도'])))
    names.append(data['카카오위치명'])
    
Mt_Hanla = [33.362500, 126.533694]
map_jeju2 = folium.Map(location = Mt_Hanla, zoom_start = 11)

marker_cluster = MarkerCluster(
    locations=locations, popups=names,
    name='jeju',
    overlay=True,
    control=True
)

marker_cluster.add_to(map_jeju2)
folium.LayerControl().add_to(map_jeju2)

map_jeju2

In [20]:
#find content with specific word
raw_total = pd.read_excel('./files/1_crawling_raw.xlsx')
raw_total.head()

content        data like   place  \
0  제주도 도착~~공항근처 #제주분식 에서 먹은 첫식사만족만족!#제주맛집#제주도맛집#제...  2019-01-01  705     NaN   
1  ﻿#함덕맛집 무거버거 바다 보면서 먹으니까 더 꿀맛👍🏻 #제주도카페투어#제주일상#함...  2019-01-01  384     NaN   
2  .대만족 제주스!.#동문시장#제주동문시장#제주도기념품#제주기념품#제주스는완벽#제주도...  2019-01-01  595     NaN   
3  #제주여행#2일차무거버거 제주와서 먹은것중에 제일 맛있었음!#제주도여행#제주맛집#함...  2019-01-01  412     NaN   
4  🥇1월 영업 안내🥇-1월 2일, 3일(수,목)은 쉬어갑니다-1월 4일,5일,6일(금...  2019-01-01  364  로타리과자점   

                                                tags  
0  ['#제주분식', '#제주맛집', '#제주도맛집', '#제주맛집추천', '#제주도맛...  
1  ['#함덕맛집', '#제주도카페투어', '#제주일상', '#함덕', '#jejudo...  
2  ['#동문시장', '#제주동문시장', '#제주도기념품', '#제주기념품', '#제주...  
3  ['#제주여행', '#2일차무거버거', '#제주도여행', '#제주맛집', '#함덕맛...  
4                                                 []

In [21]:
select_word = '해돋이'

check_list = []
for content in raw_total['content']:
    if select_word in content:
        check_list.append(True)
    else:
        check_list.append(False)
        
select_df = raw_total[check_list]
select_df.head()

content        data like  \
878   곽지스테이션안녕 반가워..2019년...붉은해의 기운을 받으세요. 얍얍얍!!!..#...  2019-01-01   31   
1552  제주에서 마지막 밥은...해물라면구름낀 날씨에 전국에서 제주빼고 다 보였다는 해돋이...  2019-01-01   13   
1626  #제주 를 제외한 전국에서 해돋이를 볼수 있....에서의 그 제주.... 포기하고 ...  2019-01-01   45   
1994  사장님 아침 대바람부터 문열어 주셔서 감사해요 ㅋㅋㅋㅋ 참 친절하셔요 또 방문 하고...  2019-01-01  114   
2109  #어부피자 #2018년 찾아주셔서 #감사합니다#2019년도 더 더욱 #노력하는 어부...  2018-12-31   35   

                 place                                               tags  
878             곽지스테이션  ['#곽지스테이션', '#곽지해수욕장', '#해돋이', '#2019', '#기해년'...  
1552               NaN  ['#제주도', '#제주도맛집', '#성산일출봉맛집', '#제주경미네', '#경미네...  
1626             광치기해변                              ['#제주', '#호텔', '#빼꼼']  
1994  Woljeongri Beach  ['#월정리', '#월정리해변', '#월정리맛집', '#밥세끼', '#제주도맛집',...  
2109               NaN  ['#어부피자', '#2018년', '#감사합니다', '#2019년도', '#노력하...